# Pizza pizza pizza

Get top venues by keyword and location, querying Google, Yelp, Foursquare

#### Motivation:
 - Gmaps and Yelp GUIs sorting and filtering are limited.
 - Query all 3 services and bring combined results into sortable table and map widget

#### Google

 - Needs a Google API key and module
 - [Create Google Cloud credentials and give access to Places APIs](https://console.cloud.google.com/google/maps-apis/credentials) (also, restrict IP or set other restrictions)
 - `conda install -c conda-forge -y gmaps`
 - put key in `apikey.txt`
 - `gmaps` Jupyter nbextension to show maps in notebook, with marker pins etc.

```
conda install -c conda-forge -y jupyter_contrib_nbextensions
jupyter nbextension enable --py gmaps
jupyter notebook
```

#### Yelp
 - needs Yelp API key and module
 - https://www.yelp.com/developers/documentation/v3
 - https://github.com/gfairchild/yelpapi
 - put key in `yelpkey.txt`
 
#### Foursquare
- Needs Foursquare API key and module
- https://developer.foursquare.com/docs/places-api/getting-started/
- https://github.com/mLewisLogic/foursquare
- OAuth id in `foursquare_id.txt`
- OAuth secret in `foursquare_secret.txt`

See `requirements.txt` for versions used, other requirements (requests, beakerx, folium, Flask)

Was going to try OpenTable and TripAdvisor but their language seems to limit API key access to approved commercial partners.
- https://dev.opentable.com/affiliate-partners/
- https://www.tripadvisor.com/APIAccessSupport

In [1]:
import time
from pprint import pprint
import ipywidgets
from ipywidgets import widgets, interact
from itertools import product
from os import path

import traceback
import pdb

import numpy as np
import pandas as pd

# cluster similar entities
import pandas_dedupe

import sklearn
from sklearn.preprocessing import StandardScaler

import requests, json 

import qgrid
from qgrid import show_grid

# convert coords to km using haversine distance 
import geopy
from geopy.distance import distance

import gmaps
with open('apikeys/apikey.txt') as f:
    api_key = f.readline().strip()
    f.close
gmaps.configure(api_key=api_key)

# https://github.com/gfairchild/yelpapi
import yelpapi
from yelpapi import YelpAPI
with open('apikeys/yelpkey.txt') as f:
    yelp_key = f.readline().strip()
    f.close
yelp_api = YelpAPI(yelp_key)

import foursquare
from foursquare import Foursquare, FoursquareException
with open('apikeys/foursquare_id.txt') as f:
    foursquare_id = f.readline().strip()
    f.close
with open('apikeys/foursquare_secret.txt') as f:
    foursquare_secret = f.readline().strip()
    f.close

# interactive maps
import folium

print("%-20s %s" % ("numpy", np.__version__))
print("%-20s %s" % ("pandas", pd.__version__))
print("%-20s %s" % ("ipywidgets", ipywidgets.__version__))
print("%-20s %s" % ("qgrid", qgrid.__version__))
print("%-20s %s" % ("sklearn", sklearn.__version__))
print("%-20s %s" % ("requests", requests.__version__))
print("%-20s %s" % ("geopy", geopy.__version__))
print("%-20s %s" % ("gmaps", gmaps.__version__))
print("%-20s %s" % ("foursquare", foursquare.__version__))
print("%-20s %s" % ("folium", folium.__version__))


numpy                1.18.1
pandas               1.0.3
ipywidgets           7.5.1
qgrid                1.3.1
sklearn              0.22.1
requests             2.23.0
geopy                2.0.0
gmaps                0.9.0
foursquare           1!2020.1.30
folium               0.11.0


## Google Maps

In [2]:
# pick a search term
keyword_options = [('Pizza', 'pizza'), ('Coffee', 'coffee'), ('Ice Cream', 'icecream')]
keyword = 'pizza'

@interact
def get_kw(kw = widgets.Dropdown(
    options=keyword_options,
    value=keyword,
    description='Search term:  ',
    disabled=False,
)):
    global keyword
    keyword = kw
    return None


interactive(children=(Dropdown(description='Search term:  ', options=(('Pizza', 'pizza'), ('Coffee', 'coffee')…

In [3]:
# pick a location
location = '40.7484, -73.9857'
location_coords = tuple(eval(location))
location_options = [('Midtown', '40.7484, -73.9857'),
                    ('Downtown', '40.7077443,-74.0139089'),
                    ('Upper East Side', '40.7711473,-73.9661166'),
                    ('Upper West Side', '40.778794,-73.984257'),
                    ('Brooklyn Heights', '40.6915812,-73.9954095'), 
                    ('Grand Army Plaza', '40.671872,-73.972544'),
                    ('Bay Ridge', '40.6292633,-74.0309554'),
                    ('Williamsburg', '40.7144609,-73.9553373'),
                  ]

@interact
def get_loc(loc = widgets.Dropdown(
    options=location_options,
    value=location,
    description='Location:',
)):
    global location
    global location_coords
    location = loc
    location_coords = tuple(eval(loc))
    return None

interactive(children=(Dropdown(description='Location:', options=(('Midtown', '40.7484, -73.9857'), ('Downtown'…

In [4]:
# pick anything we don't have a pickle file for
for test_keyword, test_location in product(keyword_options, location_options):
    keyword = test_keyword[1]
    location_name = test_location[0]
    location = test_location[1]
    location_coords = tuple(eval(test_location[1]))
    test_pickle_filename = keyword + "_" + location_name.replace(' ', '').lower() + ".pkl"
    
    if not path.exists(test_pickle_filename):
        break
        
print(location_name, keyword, location, location_coords)
print(test_pickle_filename)

Upper West Side pizza 40.778794,-73.984257 (40.778794, -73.984257)
pizza_upperwestside.pkl


In [5]:
# gmap of chosen location
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=location_coords, zoom_level=12, layout=figure_layout)
fig.add_layer(gmaps.marker_layer([location_coords]))
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [6]:
# global options for all search services APIs
MIN_USER_RATINGS = 20
MIN_RATING = 0
NRESULTS = 50
RADIUS = 1000


In [7]:
# gmaps options
GMAPS_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# https://developers.google.com/places/web-service/search#TextSearchRequests
#https://developers.google.com/places/web-service/supported_types
# rankby='prominence'
rankby='distance'
ltype='establishment'


In [8]:
def gmaps_get_first_page(api_key, location, **kwargs):
    """get first page of results from gmaps using api_key, location, kwargs for search spec"""
    # use either rankby or radius kwarg
    request_url = GMAPS_URL + '?key=' + api_key
    request_url += '&location=' + location
    for name, val in kwargs.items():
        request_url += '&' + name + '=' + str(val)
    r = requests.get(request_url)
    j = r.json()
    return j


def gmaps_get_next_page(api_key, next_page_token):
    """get next search engine results page page using search token, waiting until available"""
    r = requests.get(GMAPS_URL + '?pagetoken=' + next_page_token +
                        '&key=' + api_key)
    for i in range(10):
        j = r.json()
        if not j['results']: # wait for next page to be available
            time.sleep(5)
            continue
        else:
            return j


def gmaps_get_all_df(api_key, location, **kwargs):
    """return dataframe of all results using api_key, location, search kwargs"""
    # get first page
    j = gmaps_get_first_page(api_key, location, **kwargs)
    venues_df = pd.json_normalize(j['results'])

    # get pages while additional pages available
    while 'next_page_token' in j:
        next_page_token = j['next_page_token']
        time.sleep(5)
        j = gmaps_get_next_page(api_key, next_page_token)
        venues_df = venues_df.append(pd.json_normalize(j['results']))
        
    return venues_df


def gmaps_get_df(location_coords, keyword):

    # use either rankby or radius
    location = "%.7f,%.7f" % location_coords
    gmaps_df = gmaps_get_all_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
    if gmaps_df.empty:
        return None
    else:
        # gmaps_get_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
        gmaps_df = gmaps_df.loc[(gmaps_df['user_ratings_total'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
                           .sort_values(['rating', 'user_ratings_total'], ascending=False) \
                           .reset_index(drop=True)
        gmaps_df = gmaps_df[['name', 'vicinity', 'rating', 'user_ratings_total', 'geometry.location.lat', 'geometry.location.lng']]
        gmaps_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng']
        # drop trailing ", Brooklyn"
        gmaps_df['address'] = gmaps_df['address'].apply(lambda address: " ".join(address.split(',')[:-1]))
        gmaps_df['distance'] = gmaps_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                              axis=1)
        return gmaps_df


In [9]:
%%time
gmaps_df = gmaps_get_df(location_coords, keyword)
gmaps_df = gmaps_df.loc[(gmaps_df['nratings'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
        .sort_values(['rating', 'nratings'], ascending=False) \
        .reset_index(drop=True)
gmaps_df.to_pickle('gmaps_' + test_pickle_filename)
gmaps_df


CPU times: user 66.1 ms, sys: 8.35 ms, total: 74.5 ms
Wall time: 7.06 s


,name,address,rating,nratings,lat,lng,distance
0,My Pie Pizzeria Romana,166 W 72nd St,4.7,240,40.778269,-73.981175,0.266623
1,Pizzeria Sirenetta,568 Amsterdam Ave,4.6,315,40.788806,-73.974599,1.378695
2,IL Baretto To Go,790 11th Ave,4.6,140,40.768608,-73.992339,1.321019
3,Made in New York Pizza,421 Amsterdam Ave,4.5,481,40.783850,-73.977589,0.794961
4,Motorino,510 Columbus Ave,4.5,347,40.785687,-73.973032,1.218084
5,Fluffy's Cafe & Pizzeria,370 W 58th St,4.4,1179,40.768350,-73.985112,1.162079
6,Freddie & Pepper's,303 Amsterdam Ave,4.4,610,40.780042,-73.980393,0.354389
7,Bellini,483 Columbus Ave,4.4,211,40.784664,-73.973303,1.131328
8,Columbus Circle Pizza,10 Columbus Circle Ste Sc101,4.4,37,40.768553,-73.983187,1.140887
9,Patsy's Pizzeria,61 W 74th St,4.3,908,40.778704,-73.977457,0.574141


In [10]:
# sort grid by clicking on header (can also click on filter button)
show_grid(gmaps_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [11]:
# plot on google map

markers = [(row.lat, row.lng) for row in gmaps_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in gmaps_df.itertuples()]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Google rating</dt><dd>{rating}</dd>
<dt>Google reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**row) for i, row in gmaps_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=eval(location), zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [12]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
    
venues_map

## Yelp

In [13]:
def yelp_get_df(location_coords, keyword):
    lat, lng = location_coords
    response = yelp_api.search_query(categories=keyword, latitude=lat, longitude=lng, 
                                     radius=RADIUS, sort_by=rankby, limit=NRESULTS)

    yelp_df = pd.json_normalize(response['businesses'])
    if not yelp_df.empty:    
        yelp_df = yelp_df.loc[(yelp_df['review_count'] >= MIN_USER_RATINGS) & (yelp_df['rating'] >= MIN_RATING)] \
                         .sort_values(['rating', 'review_count'], ascending=False) \
                         .reset_index(drop=True)
        display_columns = ['name', 'location.address1', 'rating', 'review_count', 'coordinates.latitude', 'coordinates.longitude', 'url']
        yelp_df = yelp_df[display_columns]
        yelp_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        yelp_df['distance'] = yelp_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                              axis=1)
        
        if yelp_df.empty:
            return None
        else:
            return yelp_df
    else:
        return None

yelp_df = yelp_get_df(location_coords, keyword)
yelp_df

,name,address,rating,nratings,lat,lng,url,distance
0,Scott's Pizza Tours,,5.0,314,40.750244,-73.997025,https://www.yelp.com/biz/scotts-pizza-tours-ne...,3.348705
1,My Pie,166 West 72nd St,4.5,152,40.778230,-73.981170,https://www.yelp.com/biz/my-pie-new-york-5?adj...,0.268014
2,Pop's Pizzas,301 West 57th St,4.5,40,40.766990,-73.983090,https://www.yelp.com/biz/pops-pizzas-new-york?...,1.314526
3,Il Baretto To Go,790 11th Ave,4.5,33,40.768730,-73.992240,https://www.yelp.com/biz/il-baretto-to-go-new-...,1.305080
4,Pizza Collective,2060 Broadway,4.5,22,40.777950,-73.981550,https://www.yelp.com/biz/pizza-collective-new-...,0.246989
5,Freddie & Pepper's,303 Amsterdam Ave,4.0,469,40.779999,-73.980301,https://www.yelp.com/biz/freddie-and-peppers-n...,0.359759
6,Masseria Dei Vini,887 9th Ave,4.0,444,40.768400,-73.985580,https://www.yelp.com/biz/masseria-dei-vini-new...,1.159640
7,Made In New York Pizza,421 Amsterdam Ave,4.0,263,40.783841,-73.977569,https://www.yelp.com/biz/made-in-new-york-pizz...,0.795514
8,Peperino NYC Pizza,21 West End Ave,4.0,46,40.772720,-73.990430,https://www.yelp.com/biz/peperino-nyc-pizza-ne...,0.852373
9,Celeste Restaurant,502 Amsterdam Ave,3.5,897,40.786660,-73.975990,https://www.yelp.com/biz/celeste-restaurant-ne...,1.118030


In [14]:
yelp_df.to_pickle('yelp_' + test_pickle_filename)
show_grid(yelp_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [15]:
markers = [(row.lat, row.lng) for row in yelp_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in yelp_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Yelp rating</dt><dd>{rating}</dd>
<dt>Yelp reviews</dt><dd>{nratings}</dd>
</dl>
"""

marker_info = [info_box_template.format(**row) for i, row in yelp_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [16]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## Foursquare

In [17]:
def foursquare_get_df(location_coords, keyword):
    location_str = "%.7f,%.7f" % location_coords
    client = Foursquare(client_id=foursquare_id, 
                                   client_secret=foursquare_secret, 
                                   redirect_uri='http://streeteye.com/')
    response = client.venues.search(params={'query': keyword, 'll': "%s" % location_str, 
                                            'radius': RADIUS, 'limit': NRESULTS})

    foursquare_array = []

    for i, venue in pd.json_normalize(response['venues']).iterrows():
        venue_id = venue['id']
        # query detailed venue info from foursquare
        venue_name = venue['name']
        venue_address = venue['location.address']
        # sometimes no URL
        try:
            venue_url = venue['delivery.url']
        except:
            venue_url = ''
        venue_lat = venue['location.lat']
        venue_lng = venue['location.lng']
        # default these to -1
        try:
            # get rating, nratings with another API call for venue details
            venue_details = client.venues(venue_id)['venue']
            venue_rating = venue_details['rating']
            venue_nratings = venue_details['ratingSignals']
        except FoursquareException as e:
            print("Foursquare exception", type(e), str(e))
        except Exception as e:
            continue
            # sometimes no rating ... probably not popular enough
            # print(type(e), str(e))
            # print(traceback.format_exc())
            # print("No rating for %s" % venue_name)

        foursquare_array.append([venue_name, venue_address, venue_rating, venue_nratings, venue_lat, venue_lng, venue_url])
            
    foursquare_df = pd.DataFrame(foursquare_array)
    
    if len(foursquare_df) and len(foursquare_df.columns):
        foursquare_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        foursquare_df = foursquare_df.loc[(foursquare_df['nratings'] >= MIN_USER_RATINGS) & (foursquare_df['rating'] >= MIN_RATING)] \
                                     .sort_values(['rating', 'nratings'], ascending=False) \
                                     .reset_index(drop=True)
        
        foursquare_df['distance'] = foursquare_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                                        axis=1)
        if foursquare_df.empty:
            return None
        else:
            return foursquare_df
    else:
        return None

foursquare_df = foursquare_get_df(location_coords, keyword)
foursquare_df

ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f42862004da622edb85e9f2'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f42863c0c92d673f988d394'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f428658be770f153bfd3748'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f428658be770f153bfd37ba'}


Foursquare exception <class 'foursquare.FoursquareException'> Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f428658be770f153bfd3748'}


ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f42865abe770f153bfd3c14'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f4285347c1c9e09942f0b08'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f4285b310aef4346c3e333b'}


Foursquare exception <class 'foursquare.FoursquareException'> Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f4285347c1c9e09942f0b08'}


ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f4285e818a1642b1c7a6726'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f42851eec297935bfd91503'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f4285f49542ef7677106fc1'}


...

ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f42866dbe770f153bfd83f1'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f4286095c3dee61cc98fbf1'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f428474edbb5d0711920a1e'}


Foursquare exception <class 'foursquare.FoursquareException'> Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f4286095c3dee61cc98fbf1'}


ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f4285497c1c9e09942f57c0'}
ERROR:foursquare:Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f42856c3c86252883350671'}


Foursquare exception <class 'foursquare.FoursquareException'> Unknown error. meta: {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f42856c3c86252883350671'}


,name,address,rating,nratings,lat,lng,url,distance
0,Made In New York Pizza,421 Amsterdam Ave,7.9,51,40.783857,-73.977491,https://www.seamless.com/menu/made-in-new-york...,0.801446
1,Freddie & Pepper's Pizza,303 Amsterdam Ave,7.1,164,40.780089,-73.980556,NaN,0.343928
2,Little Italy Pizza,2047 Broadway,6.8,110,40.777786,-73.982276,https://www.seamless.com/menu/little-italy-piz...,0.201258
3,Big Nick's Burger & Pizza Joint Too,70 W 71st St,5.6,129,40.776627,-73.979124,https://www.seamless.com/menu/big-nicks-pizza-...,0.495623


In [18]:
show_grid(foursquare_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [19]:
if foursquare_df is not None and not foursquare_df.empty:
    foursquare_df.to_pickle('foursquare_' + test_pickle_filename)
show_grid(foursquare_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [20]:
markers = [(row.lat, row.lng) for row in foursquare_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in foursquare_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Foursquare rating</dt><dd>{rating}</dd>
<dt>Foursquare reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**d_item) for i, d_item in foursquare_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [21]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## Deduplicate and merge

In [22]:
# put everything in one big df
pd.set_option('display.max_rows', None)

gmaps_df_copy, yelp_df_copy, foursquare_df_copy = None, None, None

try:
    gmaps_df_copy = gmaps_df.copy()
    gmaps_df_copy['source'] = '0_gmaps'
except: 
    pass

try:
    yelp_df_copy = yelp_df.copy() 
    yelp_df_copy['source'] = '1_yelp'
except:
    pass

try:
    foursquare_df_copy = foursquare_df.copy()
    foursquare_df_copy['source'] = '2_foursquare'
except:
    pass


venues_df = pd.concat(list(filter(lambda df: df is not None, [gmaps_df_copy, yelp_df_copy, foursquare_df_copy]))).reset_index()
venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)

venues_df.sort_values('name')

,index,name,address,rating,nratings,lat,lng,distance,source,url,latlong
24,24,Bardolino Pizza,411 Amsterdam Ave,3.6,49,40.783486,-73.977780,0.755261,0_gmaps,NaN,"(40.78348580000001, -73.9777797)"
46,20,Bardolino Pizza 2,411 Amsterdam Ave,3.5,35,40.783530,-73.977820,0.756207,1_yelp,https://www.yelp.com/biz/bardolino-pizza-2-new...,"(40.78353, -73.97782)"
7,7,Bellini,483 Columbus Ave,4.4,211,40.784664,-73.973303,1.131328,0_gmaps,NaN,"(40.784664, -73.973303)"
40,14,Bettola,412 Amsterdam Ave,3.5,202,40.783670,-73.978160,0.747046,1_yelp,https://www.yelp.com/biz/bettola-new-york-2?ad...,"(40.78367, -73.97816)"
48,22,Big Nick's Burger & Pizza Joint Too,70 W 71st St,3.0,383,40.776536,-73.979112,0.501513,1_yelp,https://www.yelp.com/biz/big-nicks-burger-and-...,"(40.7765363632278, -73.9791117224133)"
58,3,Big Nick's Burger & Pizza Joint Too,70 W 71st St,5.6,129,40.776627,-73.979124,0.495623,2_foursquare,https://www.seamless.com/menu/big-nicks-pizza-...,"(40.776627162861516, -73.97912426572007)"
17,17,Big Nick's Pizza & Burgers,70 W 71st St #70,4.0,576,40.776454,-73.979073,0.508987,0_gmaps,NaN,"(40.7764537, -73.9790728)"
51,25,Cafe 71,2061 Broadway,2.5,135,40.778200,-73.982350,0.173972,1_yelp,https://www.yelp.com/biz/cafe-71-new-york?adju...,"(40.7782, -73.98235)"
36,10,Cafe Fiorello,1900 Broadway,3.5,770,40.771340,-73.981905,0.851243,1_yelp,https://www.yelp.com/biz/cafe-fiorello-new-yor...,"(40.77134, -73.9819051)"
16,16,Casabianca Family Italian Ristorante And Pizzeria,503 W 57th St,4.1,85,40.769478,-73.988884,1.105820,0_gmaps,NaN,"(40.76947819999999, -73.9888845)"


In [23]:
# manual fix 
# venues_df.loc[venues_df['address']=='212b Pacific St', 'address'] = '212 Pacific St'
# venues_df

In [24]:
# run dedupe algorithm using name, address as default texts, latlong as latlong
venues_df['shortname'] = venues_df['name'].apply(lambda n: n[:25])
venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['name', 'address', ('latlong', 'LatLong')])


Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 37


In [25]:
# view clustering
venues_df['cluster'] = venues_df2['cluster id']
venues_df = venues_df.sort_values(['cluster', 'source'])[['cluster', 'name', 'address', 'rating', 'nratings', 'lat', 'lng', 'source']]
venues_df

,cluster,name,address,rating,nratings,lat,lng,source
0,0,My Pie Pizzeria Romana,166 W 72nd St,4.7,240,40.778269,-73.981175,0_gmaps
27,0,My Pie,166 West 72nd St,4.5,152,40.77823,-73.98117,1_yelp
2,1,IL Baretto To Go,790 11th Ave,4.6,140,40.7686076,-73.9923386,0_gmaps
29,1,Il Baretto To Go,790 11th Ave,4.5,33,40.76873,-73.99224,1_yelp
3,2,Made in New York Pizza,421 Amsterdam Ave,4.5,481,40.7838496,-73.9775895,0_gmaps
33,2,Made In New York Pizza,421 Amsterdam Ave,4.0,263,40.783841,-73.977569,1_yelp
55,2,Made In New York Pizza,421 Amsterdam Ave,7.9,51,40.783857,-73.977491,2_foursquare
6,3,Freddie & Pepper's,303 Amsterdam Ave,4.4,610,40.78004190000001,-73.9803931,0_gmaps
31,3,Freddie & Pepper's,303 Amsterdam Ave,4.0,469,40.779999,-73.980301,1_yelp
56,3,Freddie & Pepper's Pizza,303 Amsterdam Ave,7.1,164,40.78008888335382,-73.9805559440812,2_foursquare


In [26]:
# group by clusters
cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'source']] \
    .first() \
    .reset_index()
cluster_df

,cluster,name,address,lat,lng,source
0,0,My Pie Pizzeria Romana,166 W 72nd St,40.778269,-73.981175,0_gmaps
1,1,IL Baretto To Go,790 11th Ave,40.7686076,-73.9923386,0_gmaps
2,2,Made in New York Pizza,421 Amsterdam Ave,40.7838496,-73.9775895,0_gmaps
3,3,Freddie & Pepper's,303 Amsterdam Ave,40.78004190000001,-73.9803931,0_gmaps
4,4,Patsy's Pizzeria,61 W 74th St,40.7787036,-73.97745669999999,0_gmaps
5,5,Celeste Restaurant,502 Amsterdam Ave,40.7866992,-73.9760929,0_gmaps
6,6,Marinara Pizza Upper West,483 Amsterdam Ave,40.785767,-73.976047,0_gmaps
7,7,Columbus Gourmet Food,261 Columbus Ave,40.777546,-73.978487,0_gmaps
8,8,La Traviata Pizzeria,101 W 68th St,40.7751046,-73.9809475,0_gmaps
9,9,Casabianca Family Italian Ristorante And Pizzeria,503 W 57th St,40.76947819999999,-73.9888845,0_gmaps


In [27]:
# make markers on clusters
# add ratings on all rows

markers = [(float(row.lat), float(row.lng)) for row in cluster_df.itertuples()]
marker_hover = ["%s" % (row.name) for row in cluster_df.itertuples()]

# make a dict by cluster, initialize rating string to ''
marker_dict = {i: {'name': row['name'],
                   'address': row['address'],
                   'lat': row['lat'],
                   'lng': row['lng'],
                   'rate_str': '',
                  } for i, row in cluster_df.iterrows()}

# add all rating strings
for i, row in venues_df.iterrows():
    cluster = row['cluster']
    if row.source[2:] == 'gmaps':
        marker_dict[cluster]['rate_str'] += "<dt>Google rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'yelp':
        marker_dict[cluster]['rate_str'] += "<dt>Yelp rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'foursquare':
        marker_dict[cluster]['rate_str'] += "<dt>Foursquare rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
{rate_str}

</dl>
"""    

marker_info = [info_box_template.format(**d_item) for i, d_item in marker_dict.items()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [28]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map


In [29]:
venues_df

,cluster,name,address,rating,nratings,lat,lng,source
0,0,My Pie Pizzeria Romana,166 W 72nd St,4.7,240,40.778269,-73.981175,0_gmaps
27,0,My Pie,166 West 72nd St,4.5,152,40.77823,-73.98117,1_yelp
2,1,IL Baretto To Go,790 11th Ave,4.6,140,40.7686076,-73.9923386,0_gmaps
29,1,Il Baretto To Go,790 11th Ave,4.5,33,40.76873,-73.99224,1_yelp
3,2,Made in New York Pizza,421 Amsterdam Ave,4.5,481,40.7838496,-73.9775895,0_gmaps
33,2,Made In New York Pizza,421 Amsterdam Ave,4.0,263,40.783841,-73.977569,1_yelp
55,2,Made In New York Pizza,421 Amsterdam Ave,7.9,51,40.783857,-73.977491,2_foursquare
6,3,Freddie & Pepper's,303 Amsterdam Ave,4.4,610,40.78004190000001,-73.9803931,0_gmaps
31,3,Freddie & Pepper's,303 Amsterdam Ave,4.0,469,40.779999,-73.980301,1_yelp
56,3,Freddie & Pepper's Pizza,303 Amsterdam Ave,7.1,164,40.78008888335382,-73.9805559440812,2_foursquare


In [30]:
merge_df = cluster_df \
    .merge(venues_df.loc[venues_df['source']=='0_gmaps'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'gmaps_rating', 'nratings': 'gmaps_nratings'})
merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='1_yelp'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'yelp_rating', 'nratings': 'yelp_nratings'})
merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='2_foursquare'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'foursquare_rating', 'nratings': 'foursquare_nratings'})
merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

merge_df 

,cluster,name,address,lat,lng,source,gmaps_rating,gmaps_nratings,gmaps_rating_std,yelp_rating,yelp_nratings,yelp_rating_std,foursquare_rating,foursquare_nratings,foursquare_rating_std
0,0,My Pie Pizzeria Romana,166 W 72nd St,40.778269,-73.981175,0_gmaps,4.7,240,1.614000,4.5,152,1.407022,NaN,NaN,NaN
1,1,IL Baretto To Go,790 11th Ave,40.7686076,-73.9923386,0_gmaps,4.6,140,1.307694,4.5,33,1.407022,NaN,NaN,NaN
2,2,Made in New York Pizza,421 Amsterdam Ave,40.7838496,-73.9775895,0_gmaps,4.5,481,1.001387,4.0,263,0.665138,7.9,51,1.270978
3,3,Freddie & Pepper's,303 Amsterdam Ave,40.78004190000001,-73.9803931,0_gmaps,4.4,610,0.695080,4.0,469,0.665138,7.1,164,0.302614
4,4,Patsy's Pizzeria,61 W 74th St,40.7787036,-73.97745669999999,0_gmaps,4.3,908,0.388774,3.5,503,-0.076747,NaN,NaN,NaN
5,5,Celeste Restaurant,502 Amsterdam Ave,40.7866992,-73.9760929,0_gmaps,4.3,443,0.388774,3.5,897,-0.076747,NaN,NaN,NaN
6,6,Marinara Pizza Upper West,483 Amsterdam Ave,40.785767,-73.976047,0_gmaps,4.3,135,0.388774,3.5,70,-0.076747,NaN,NaN,NaN
7,7,Columbus Gourmet Food,261 Columbus Ave,40.777546,-73.978487,0_gmaps,4.1,233,-0.223839,2.5,94,-1.560516,NaN,NaN,NaN
8,8,La Traviata Pizzeria,101 W 68th St,40.7751046,-73.9809475,0_gmaps,4.1,147,-0.223839,3.5,175,-0.076747,NaN,NaN,NaN
9,9,Casabianca Family Italian Ristorante And Pizzeria,503 W 57th St,40.76947819999999,-73.9888845,0_gmaps,4.1,85,-0.223839,3.0,53,-0.818631,NaN,NaN,NaN


In [31]:
merge_df['distance'] = merge_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                      axis=1)
merge_df


,cluster,name,address,lat,lng,source,gmaps_rating,gmaps_nratings,gmaps_rating_std,yelp_rating,yelp_nratings,yelp_rating_std,foursquare_rating,foursquare_nratings,foursquare_rating_std,distance
0,0,My Pie Pizzeria Romana,166 W 72nd St,40.778269,-73.981175,0_gmaps,4.7,240,1.614000,4.5,152,1.407022,NaN,NaN,NaN,0.266623
1,1,IL Baretto To Go,790 11th Ave,40.7686076,-73.9923386,0_gmaps,4.6,140,1.307694,4.5,33,1.407022,NaN,NaN,NaN,1.321019
2,2,Made in New York Pizza,421 Amsterdam Ave,40.7838496,-73.9775895,0_gmaps,4.5,481,1.001387,4.0,263,0.665138,7.9,51,1.270978,0.794961
3,3,Freddie & Pepper's,303 Amsterdam Ave,40.78004190000001,-73.9803931,0_gmaps,4.4,610,0.695080,4.0,469,0.665138,7.1,164,0.302614,0.354389
4,4,Patsy's Pizzeria,61 W 74th St,40.7787036,-73.97745669999999,0_gmaps,4.3,908,0.388774,3.5,503,-0.076747,NaN,NaN,NaN,0.574141
5,5,Celeste Restaurant,502 Amsterdam Ave,40.7866992,-73.9760929,0_gmaps,4.3,443,0.388774,3.5,897,-0.076747,NaN,NaN,NaN,1.116050
6,6,Marinara Pizza Upper West,483 Amsterdam Ave,40.785767,-73.976047,0_gmaps,4.3,135,0.388774,3.5,70,-0.076747,NaN,NaN,NaN,1.039178
7,7,Columbus Gourmet Food,261 Columbus Ave,40.777546,-73.978487,0_gmaps,4.1,233,-0.223839,2.5,94,-1.560516,NaN,NaN,NaN,0.506417
8,8,La Traviata Pizzeria,101 W 68th St,40.7751046,-73.9809475,0_gmaps,4.1,147,-0.223839,3.5,175,-0.076747,NaN,NaN,NaN,0.495897
9,9,Casabianca Family Italian Ristorante And Pizzeria,503 W 57th St,40.76947819999999,-73.9888845,0_gmaps,4.1,85,-0.223839,3.0,53,-0.818631,NaN,NaN,NaN,1.105820


In [32]:
# simple average score
merge_df['meanrating'] = np.nanmean(merge_df[['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']], axis=1)
merge_df.sort_values('meanrating', ascending=False)[['name', 'address', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'meanrating']]


,name,address,gmaps_rating,yelp_rating,foursquare_rating,meanrating
27,Scott's Pizza Tours,,NaN,5.0,NaN,2.148907
0,My Pie Pizzeria Romana,166 W 72nd St,4.7,4.5,NaN,1.510511
29,Pizza Collective,2060 Broadway,NaN,4.5,NaN,1.407022
28,Pop's Pizzas,301 West 57th St,NaN,4.5,NaN,1.407022
1,IL Baretto To Go,790 11th Ave,4.6,4.5,NaN,1.357358
18,Pizzeria Sirenetta,568 Amsterdam Ave,4.6,NaN,NaN,1.307694
19,Motorino,510 Columbus Ave,4.5,NaN,NaN,1.001387
2,Made in New York Pizza,421 Amsterdam Ave,4.5,4.0,7.9,0.979168
22,Columbus Circle Pizza,10 Columbus Circle Ste Sc101,4.4,NaN,NaN,0.695080
20,Fluffy's Cafe & Pizzeria,370 W 58th St,4.4,NaN,NaN,0.695080


In [33]:
# bayes score
rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
nratings_mean = np.mean(merge_df['nratings'])
rating_avg = np.nanmean(merge_df[rating_cols])
merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
print('mean number of ratings', nratings_mean)
print('average rating', rating_avg)
merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg

show_grid(merge_df.sort_values('bayes_score', ascending=False)[['name', 'address', 'distance', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'nratings', 'bayes_score']])

mean number of ratings 1.5789473684210527
average rating -0.01904598733964774


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [34]:
def dedupe(dedupe_list):

    for i, source_df in enumerate(dedupe_list):
        source_df['source'] = i
    venues_df = pd.concat(dedupe_list).reset_index()
    venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)
    venues_df['shortname'] = venues_df['name'].apply(lambda n: n[:25])

    # dedupe and assign cluster id
    venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['shortname', 'address', ('latlong', 'LatLong')])
    venues_df['cluster'] = venues_df2['cluster id']
    venues_df = venues_df.sort_values(['cluster', 'source'])[['cluster', 'name', 'address', 'rating', 'nratings', 'lat', 'lng', 'distance', 'source']]

    # group by clusters, uniquify name
    cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'distance', 'source']] \
                          .first() \
                          .reset_index()

    # merge ratings by source
    merge_df = cluster_df \
        .merge(venues_df.loc[venues_df['source']=='0'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
        .rename(columns={'rating': 'gmaps_rating', 'nratings': 'gmaps_nratings'})
    merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='1'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
        .rename(columns={'rating': 'yelp_rating', 'nratings': 'yelp_nratings'})
    merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='2'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
        .rename(columns={'rating': 'foursquare_rating', 'nratings': 'foursquare_nratings'})
    merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

    # bayes score
    rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
    merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
    nratings_mean = np.mean(merge_df['nratings'])
    rating_avg = np.nanmean(merge_df[rating_cols])
    merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
    merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
    merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg
    merge_df = merge_df.sort_values('bayes_score', ascending=False)
    return merge_df

    
dedupe_list = list(filter(lambda df: df is not None, [gmaps_df, yelp_df, foursquare_df]))

dedupe_df = dedupe(dedupe_list)
dedupe_df

Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 37


,cluster,name,address,lat,lng,source,gmaps_rating,gmaps_nratings,gmaps_rating_std,yelp_rating,yelp_nratings,yelp_rating_std,foursquare_rating,foursquare_nratings,foursquare_rating_std,nratings,w,R,bayes_score
0,0,My Pie Pizzeria Romana,166 W 72nd St,40.778269,-73.981175,0,4.7,240,1.614000,4.5,152,1.407022,NaN,NaN,NaN,2,0.558824,1.510511,0.835707
27,26,Scott's Pizza Tours,,40.750244140625,-73.9970245361328,1,NaN,NaN,NaN,5.0,314,2.148907,NaN,NaN,NaN,1,0.387755,2.148907,0.821589
1,1,IL Baretto To Go,790 11th Ave,40.7686076,-73.9923386,0,4.6,140,1.307694,4.5,33,1.407022,NaN,NaN,NaN,2,0.558824,1.357358,0.750121
2,2,Made in New York Pizza,421 Amsterdam Ave,40.7838496,-73.9775895,0,4.5,481,1.001387,4.0,263,0.665138,7.9,51,1.270978,3,0.655172,0.979168,0.634956
29,28,Pizza Collective,2060 Broadway,40.77795,-73.98155,1,NaN,NaN,NaN,4.5,22,1.407022,NaN,NaN,NaN,1,0.387755,1.407022,0.533919
28,27,Pop's Pizzas,301 West 57th St,40.76699,-73.98309,1,NaN,NaN,NaN,4.5,40,1.407022,NaN,NaN,NaN,1,0.387755,1.407022,0.533919
18,17,Pizzeria Sirenetta,568 Amsterdam Ave,40.7888061,-73.97459890000002,0,4.6,315,1.307694,NaN,NaN,NaN,NaN,NaN,NaN,1,0.387755,1.307694,0.495404
19,18,Motorino,510 Columbus Ave,40.785687,-73.973032,0,4.5,347,1.001387,NaN,NaN,NaN,NaN,NaN,NaN,1,0.387755,1.001387,0.376632
3,3,Freddie & Pepper's,303 Amsterdam Ave,40.78004190000001,-73.9803931,0,4.4,610,0.695080,4.0,469,0.665138,7.1,164,0.302614,3,0.655172,0.554277,0.356580
20,19,Fluffy's Cafe & Pizzeria,370 W 58th St,40.7683497,-73.985112,0,4.4,1179,0.695080,NaN,NaN,NaN,NaN,NaN,NaN,1,0.387755,0.695080,0.257860


In [35]:
dedupe_df.to_pickle(test_pickle_filename)
pd.read_pickle(test_pickle_filename)

,cluster,name,address,lat,lng,source,gmaps_rating,gmaps_nratings,gmaps_rating_std,yelp_rating,yelp_nratings,yelp_rating_std,foursquare_rating,foursquare_nratings,foursquare_rating_std,nratings,w,R,bayes_score
0,0,My Pie Pizzeria Romana,166 W 72nd St,40.778269,-73.981175,0,4.7,240,1.614000,4.5,152,1.407022,NaN,NaN,NaN,2,0.558824,1.510511,0.835707
27,26,Scott's Pizza Tours,,40.750244140625,-73.9970245361328,1,NaN,NaN,NaN,5.0,314,2.148907,NaN,NaN,NaN,1,0.387755,2.148907,0.821589
1,1,IL Baretto To Go,790 11th Ave,40.7686076,-73.9923386,0,4.6,140,1.307694,4.5,33,1.407022,NaN,NaN,NaN,2,0.558824,1.357358,0.750121
2,2,Made in New York Pizza,421 Amsterdam Ave,40.7838496,-73.9775895,0,4.5,481,1.001387,4.0,263,0.665138,7.9,51,1.270978,3,0.655172,0.979168,0.634956
29,28,Pizza Collective,2060 Broadway,40.77795,-73.98155,1,NaN,NaN,NaN,4.5,22,1.407022,NaN,NaN,NaN,1,0.387755,1.407022,0.533919
28,27,Pop's Pizzas,301 West 57th St,40.76699,-73.98309,1,NaN,NaN,NaN,4.5,40,1.407022,NaN,NaN,NaN,1,0.387755,1.407022,0.533919
18,17,Pizzeria Sirenetta,568 Amsterdam Ave,40.7888061,-73.97459890000002,0,4.6,315,1.307694,NaN,NaN,NaN,NaN,NaN,NaN,1,0.387755,1.307694,0.495404
19,18,Motorino,510 Columbus Ave,40.785687,-73.973032,0,4.5,347,1.001387,NaN,NaN,NaN,NaN,NaN,NaN,1,0.387755,1.001387,0.376632
3,3,Freddie & Pepper's,303 Amsterdam Ave,40.78004190000001,-73.9803931,0,4.4,610,0.695080,4.0,469,0.665138,7.1,164,0.302614,3,0.655172,0.554277,0.356580
20,19,Fluffy's Cafe & Pizzeria,370 W 58th St,40.7683497,-73.985112,0,4.4,1179,0.695080,NaN,NaN,NaN,NaN,NaN,NaN,1,0.387755,0.695080,0.257860


In [7]:
locationdict = dict([('midtown', '40.7484, -73.9857'),
                     ('downtown', '40.7077443,-74.0139089'),
                     ('uppereastside', '40.7711473,-73.9661166'),
                     ('upperwestside', '40.778794,-73.984257'),
                     ('brooklynheights', '40.6915812,-73.9954095'), 
                     ('grandarmyplaza', '40.671872,-73.972544'),
                     ('bayridge', '40.6292633,-74.0309554'),
                     ('williamsburg', '40.7144609,-73.9553373')])
locationdict

{'midtown': '40.7484, -73.9857',
 'downtown': '40.7077443,-74.0139089',
 'uppereastside': '40.7711473,-73.9661166',
 'upperwestside': '40.778794,-73.984257',
 'brooklynheights': '40.6915812,-73.9954095',
 'grandarmyplaza': '40.671872,-73.972544',
 'bayridge': '40.6292633,-74.0309554',
 'williamsburg': '40.7144609,-73.9553373'}

In [9]:

for k, l in product(['pizza', 'coffee', 'icecream'], 
                    ['midtown','downtown','uppereastside','upperwestside','brooklynheights','grandarmyplaza','bayridge','williamsburg',]):
    filename = "cache/%s_%s.pkl" % (k, l)
    location_coords = eval(locationdict[l])
    tempdf = pd.read_pickle(filename)
    tempdf['distance'] = tempdf.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                      axis=1)
    tempdf.to_pickle(filename)
    print(filename)

cache/pizza_midtown.pkl
cache/pizza_downtown.pkl
cache/pizza_uppereastside.pkl
cache/pizza_upperwestside.pkl
cache/pizza_brooklynheights.pkl
cache/pizza_grandarmyplaza.pkl
cache/pizza_bayridge.pkl
cache/pizza_williamsburg.pkl
cache/coffee_midtown.pkl
cache/coffee_downtown.pkl
cache/coffee_uppereastside.pkl
cache/coffee_upperwestside.pkl
cache/coffee_brooklynheights.pkl
cache/coffee_grandarmyplaza.pkl
cache/coffee_bayridge.pkl
cache/coffee_williamsburg.pkl
cache/icecream_midtown.pkl
cache/icecream_downtown.pkl
cache/icecream_uppereastside.pkl
cache/icecream_upperwestside.pkl
cache/icecream_brooklynheights.pkl
cache/icecream_grandarmyplaza.pkl
cache/icecream_bayridge.pkl
cache/icecream_williamsburg.pkl


In [36]:
# # file to retrain with train_dedupe2.py
# can train better if we add the keyword, not yet implemented
# tempdf = None
# for k, l, j in product(['pizza', 'coffee', 'icecream'], 
#                     ['midtown','downtown','uppereastside','upperwestside','brooklynheights','grandarmyplaza','bayridge','williamsburg',],
#                     ['gmaps', 'yelp', 'foursquare']):
#     filename = "cache/%s_%s_%s.pkl" % (j, k, l)
#     try:
#         if tempdf is None:
#             tempdf = pd.read_pickle(filename)
#             tempdf['keyword']=k
#             print(filename)
#         else:
#             tempdf2 = pd.read_pickle(filename)
#             tempdf2['keyword']=k
#             tempdf = pd.concat([tempdf, tempdf2])
#             print(filename)
#     except:
#         print("missing ", filename)